# Ejercicio 10

En una clínica de medicina nuclear se prepara a un paciente para la obtención de
imágenes. Para ello se separa un volumen de 5 ml de solución con N-13 y se mide una
concentración de actividad de 17 mCi/ml. Quince minutos después se administra a un
paciente 3 ml de este preparado.

1. ¿Cuánta masa de N-13 ingresa al paciente?
2. Si se esperan 10 minutos antes de comenzar a tomar imágenes para que el radionucleido sea captado en el órgano que se desea ver, cuánta actividad quedará disponible en total en el paciente?
3. Si cada imagen requiere 30 segundos, y por ubicación y geometría del detector sólo llega el 5% de las emisiones, ¿cuántas detecciones se producen para formar la imagen?

## Parte 1

## Actividad de la muestra

Se inyectan 3 ml de solución con actividad de 17 mCi/ml. Primero calcularemos la actividad total administrada al paciente.

In [1]:
from pint import UnitRegistry
import numpy as np
u = UnitRegistry()
u.load_definitions('./mis-defs.txt')    # carga de definiciones. agregamos la relación
                                        # entre Curies y 1/tiempo.

def mostrar(numero,precision=2):
    '''Esta función permite darle formato científico a un número. El argumento
    *precision* es el número de decimales con que se verá.
    '''    
    print( "{:.{}e}".format(numero, precision ) )

In [2]:
## Primera parte

In [3]:
vol = 3 * u('ml')
a_esp = 17 * u('mCi/ml')
A0 = vol * a_esp
A0

51.0 <Unit('millicurie')>

## Masa de N-13

Para calcular la masa, se necesita saber el número de núcleos. Conocida la actividad, despejamos en ese mismo instante el número de nucleos (sólo se necesita buscar la constante de desintegración del radionucleido):

$$N_0 = A_0 / \lambda$$



In [4]:
tmedio_n13 = 9.965 * u('min')
lmbd_n13 = np.log(2)/tmedio_n13
lmbd_n13

0.0695581716567933 <Unit('1 / minute')>

In [5]:
N0 = A0 / lmbd_n13
N0

733.1992602053846 <Unit('millicurie * minute')>

**Al mostrar N0 vemos que se podrían simplicar las unidades, ya que Ci es equivalente a una frecuencia (1/tiempo).**

In [6]:
mostrar(N0.to(''))

1.63e+12 dimensionless


Para calcular la masa correspondiente a este número de átomos, tendremos que aplicar una regla de 3:

| atomos | masa |
| ------ | ---- |
| 6.023e23 atomos | 13 grs |
| 2.71e+12 atomos | x |

In [7]:
m_mol = 13 * u('g')
n_avogadro = 6.023e23
mass_N13 = N0 * m_mol / n_avogadro
mostrar(mass_N13.to('pg'))

3.51e+01 picogram


## Parte 2. Cálculo de la actividad tras 10 minutos.

La actividad inyectada al paciente va decayendo "físicamente", según la siguiente ecuación:
    
$$ A(t) = A_0 \cdot e^{- \lambda t}$$

In [8]:
import math

t = 10 * u('min')
A = A0 * math.exp(- lmbd_n13 * t)
A

25.437994839319693 <Unit('millicurie')>

## Parte 3. Cantidad de detecciones.

Si cada imagen requiere 30 segundos, y por ubicación y geometría del detector sólo llega el 5% de las emisiones, ¿cuántas detecciones se producen para formar la imagen?

![Nitrógeno 13](./imagenes/n13.png)

El N-13 decae principalmente emitiendo partículas beta y fotones de aniquilación, y también por captura electrónica. Prácticamente cada nucleo decae emitiendo dos fotones de aniquilación. Para saber cuántos fotones se emitieron en 30 segundos tenemos dos opciones:

1. cálculo "exacto": primero calcular el nro de nucleos de N-13 al inicio, restar la cantidad de nucleos del mismo radionucleido tras 30 segundos, y multiplicar el resultado por la frecuencia de emisión de los fotones. 
2. cálculo "aproximado": como 30 segundos es mucho menor que el semiperíodo de desintegración (9 minutos), podríamos suponer que la actividad es aproximadamente constante, y multiplicar A0*tiempo, para obtener el número de nucleos que decayeron en ese lapso, y finalmente multiplicar por la frecuencia de emisión de fotones.

### Cálculo exacto

In [9]:
t1 = 30 * u('s')
Nf = N0 * math.exp(- lmbd_n13 * t1)
Nf.to('')

1572065465789.894 <Unit('dimensionless')>

In [10]:
deltaN_exacto = N0-Nf
mostrar(deltaN_exacto.to(''))

5.56e+10 dimensionless


### Cálculo aproximado

In [11]:
deltaN_aprox = A0 * t1
mostrar(deltaN_aprox.to(''))

5.66e+10 dimensionless


In [12]:
error_porcentual = (deltaN_exacto - deltaN_aprox) / deltaN_exacto
mostrar(error_porcentual)

-1.75e-02 dimensionless


El error porcentual resulta ser menor al 2% (que sería: 2e-2).

Hasta ahora solamente calculamos la cantidad de nucleos que decayeron de N-13 a C-13, falta considerar la frecuencia de emisión de los fotones (de aniquilación) para saber cuántos se produjeron.

Por cada decaimiento se emitieron 2 fotones de aniquilación (cada uno de ellos a 180 grados del otro). Por la geometría del problema, solo uno de estos dos fotones llegará al detector. Y también por la geometría del problema, sólo el 5% de éstos serán detectados. 

In [13]:
detecciones = 0.05 * deltaN_exacto * 1      # sólo uno de los fotones puede salir 
                                            # en dirección al detector

In [14]:
detecciones.to('')

2781844593.302983 <Unit('dimensionless')>

In [15]:
mostrar(detecciones.to(''))

2.78e+09 dimensionless
